# Clean the text

In [1]:
from __future__ import division

import re
import string
import cPickle
import unicodedata
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

Read in the pickled text and put into a dataframe.

In [2]:
fin_name = './questions.pkl'

with open(fin_name, 'r') as f_in:
    text = cPickle.load(f_in)

df = pd.DataFrame()
df['text'] = text
df.head()

,text
0,"To ask Mr Chancellor of the Exchequer, whether..."
1,"To ask Mr Chancellor of the Exchequer, how man..."
2,To ask Her Majesty's Government whether an Eng...
3,"To ask Mr Chancellor of the Exchequer, how man..."
4,"To ask Mr Chancellor of the Exchequer, when he..."


Define a list of project specific stopwords.

In [3]:
my_stopwords = ['mr', 'ask', 'chancellor', 'exchequer']

Define a function to clean and tokenise the results.

In [4]:
def tokeniser(text):
    
    # Remove any whitespace at the start and end of the string
    # and remove any stray tabs and newline characters
    text = text.strip()
    
    # Remove any weird unicode characters
    if isinstance(text, unicode):
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore')
        
    # Convert hyphens and slashes to spaces
    text = re.sub(r'[-/]+',' ',text)
    
    # Remove remaining punctuation
    text = text.translate(None, string.punctuation)
    
    # Convert the text to lowercase and use nltk tokeniser
    tokens = word_tokenize(text.lower())
    
    # Define a list of stopwords apart from the word 'not'
    stops = set(stopwords.words('english') + my_stopwords) - set(('not'))

    # Define stemmer
    stemmer = SnowballStemmer('english')

    return [stemmer.stem(i) for i in tokens if i not in stops]


Apply this function.

In [5]:
df['tokens'] = df['text'].apply(tokeniser)
df.head()

,text,tokens
0,"To ask Mr Chancellor of the Exchequer, whether...","[whether, rate, corpor, tax, paid, privat, sec..."
1,"To ask Mr Chancellor of the Exchequer, how man...","[mani, credit, union, subject, intervent, fina..."
2,To ask Her Majesty's Government whether an Eng...,"[majesti, govern, whether, english, coupl, mar..."
3,"To ask Mr Chancellor of the Exchequer, how man...","[mani, proport, depart, answer, written, parli..."
4,"To ask Mr Chancellor of the Exchequer, when he...","[plan, publish, data, except, two, child, tax,..."


Save the dataframe.

In [6]:
fout_name = "./tokenised.pkl"
df.to_pickle(fout_name)